In [1]:
!pip install dbt-core


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install dbt-bigquery


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!dbt --version

Core:
  - installed: 1.4.6
  - latest:    1.4.6 - Up to date!

Plugins:
  - bigquery: 1.4.3 - Up to date!
